# LIGHTNING IN 2 STEPS
### 본 문서는 PyTorch Lightning의 [공식 가이드](https://pytorch-lightning.readthedocs.io/en/latest/starter/new-project.html#lightning-in-2-steps)의 한글 번역본입니다. (옮긴이 [dnap512](https://github.com/dnap512), 21.7.13)
**이번 가이드에서는 PyTorch 코드를 Lightning으로 어떻게 구성할 수 있는지 2 단계로 설명합니다.**

PyTorch Lightning으로 코드를 구성하면 코드를 다음과 같이 만들 수 있습니다.

- PyTorch의 유연성을 유지하면서도, 반복적으로 비슷한 형태를 띄는 코드 무더기(a ton of boilerplate code)를 없앨 수 있습니다.
- 엔지니어링에서 연구를 위한 코드를 분리하여 가독성을 높일 수 있습니다.
- 재현이 더 쉬워집니다.
- Training loop와 까다로운 엔지니어링 코드 대부분을 자동화하여 오류를 감소시킵니다.
- 모델 변경없이 어떠한 하드웨어 조건에서도 Scalable 합니다.

## Step 0: PyTorch Lightning 설치

pip를 이용하여 설치하거나,
```bash
pip install pytorch-lightning
```

conda를 이용하여 설치할 수 있습니다.
```bash
conda install pytorch-lightning -c conda-forge
```

혹은 conda 환경에서도 설치할 수 있습니다.
```bash
conda activate my_env
pip install pytorch-lightning
```

## Step 1: LightningModule 정의

In [ ]:
import os
import torch
from torch import nn
import torch.nn.functional as F
from torchvision import transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader, random_split
import pytorch_lightning as pl

In [ ]:
class LitAutoEncoder(pl.LightningModule):

    def __init__(self):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(28*28, 64),
            nn.ReLU(),
            nn.Linear(64, 3)
        )
        self.decoder = nn.Sequential(
            nn.Linear(3, 64),
            nn.ReLU(),
            nn.Linear(64, 28*28)
        )

    def forward(self, x):
        # lightning에서는, forward 함수는 prediction/inference action을 정의합니다.
        embedding = self.encoder(x)
        return embedding

    def training_step(self, batch, batch_idx):
        # training_step은 train loop를 정의합니다.
        # forward 함수와는 독립적입니다.
        x, y = batch
        x = x.view(x.size(0), -1)
        z = self.encoder(x)
        x_hat = self.decoder(z)
        loss = F.mse_loss(x_hat, x)
        # 기본적으로 Tensorboard에 logging합니다.
        self.log('train_loss', loss)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

### System과 Model

[LightningModule](https://pytorch-lightning.readthedocs.io/en/latest/common/lightning_module.html)은 모델이 아니라 시스템을 정의합니다. (아래 그림을 참고하세요)
![module](https://pl-bolts-doc-images.s3.us-east-2.amazonaws.com/pl_docs/model_system.png)


시스템의 예:
- [Autoencoder](https://github.com/PyTorchLightning/lightning-bolts/blob/master/pl_bolts/models/autoencoders/basic_ae/basic_ae_module.py)
- [BERT](https://colab.research.google.com/github/PytorchLightning/pytorch-lightning/blob/master/notebooks/04-transformers-text-classification.ipynb)
- [DQN](https://colab.research.google.com/github/PytorchLightning/pytorch-lightning/blob/master/notebooks/08-Domain-specific-demos.ipynb)
- [GAN](https://colab.research.google.com/github/PytorchLightning/pytorch-lightning/blob/master/notebooks/03-basic-gan.ipynb)
- [Image classifier](https://colab.research.google.com/github/PytorchLightning/pytorch-lightning/blob/master/notebooks/01-mnist-hello-world.ipynb)
- Seq2seq
- [SimCLR](https://github.com/PyTorchLightning/lightning-bolts/blob/master/pl_bolts/models/self_supervised/simclr/simclr_module.py)
- [VAE](https://github.com/PyTorchLightning/lightning-bolts/blob/master/pl_bolts/models/autoencoders/basic_vae/basic_vae_module.py)

내부에서 LightningModule은 여전히 모든 연구 코드를 단일 파일로 그룹화하여 독립적으로 만드는 torch.nn.Module에 불과합니다:

- The Train loop
- The Validation loop
- The Test loop
- The Model or system of Models
- The Optimizer

여러분은 사용 가능한 Callback hooks에서 찾을 수 있는 20개 이상의 hooks를 재정의하여 Training의 모든 부분(예: Backward pass)을 Customizing할 수 있습니다.

In [ ]:
class LitAutoEncoder(LightningModule):

    def backward(self, loss, optimizer, optimizer_idx):
        loss.backward()

### FORWARD와 TRAINING_STEP

Lightning에서는 Training과 Inference를 분리합니다. Training_step은 전체 훈련 루프를 정의합니다. 우리는 사용자가 Inference 작업을 정의하기 위해 `forward`를 사용할 것을 권장합니다.

예를 들어, 이 경우 임베딩 추출기로 작동하도록 오토인코더를 정의할 수 있습니다.

In [ ]:
def forward(self, x):
    embeddings = self.encoder(x)
    return embeddings

당연하게도, `training_step` 내에서도 `forward` 함수를 사용할 수 있습니다.

In [ ]:
def training_step(self, batch, batch_idx):
    ...
    z = self(x)

이러한 것들은 정말로 여러분의 구현에 따라 달라집니다. 그러나 다음 사항들은 유지하는 것이 좋습니다.

- Inference (predicting)를 목적으로 `forward` 사용
- Training을 목적으로 `training_step` 사용

자세한 것들은 [lightning module](https://pytorch-lightning.readthedocs.io/en/latest/common/lightning_module.html) docs를 참고하세요.


## Step 2: Fit with Lightning Trainer

먼저 원하는 대로 데이터를 정의합니다. Lightning은 train/val/test 분할을 위한 [DataLoader](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader)만 있으면 됩니다.

In [ ]:
dataset = MNIST(os.getcwd(), download=True, transform=transforms.ToTensor())
train_loader = DataLoader(dataset)

다음으로, [Lightning module](https://pytorch-lightning.readthedocs.io/en/latest/common/lightning_module.html)과 PyTorch Lightning `Trainer`를 초기화한 다음 데이터와 모델 모두에 적합하게 호출합니다.

In [ ]:
# init model
autoencoder = LitAutoEncoder()

# most basic trainer, uses good defaults (auto-tensorboard, checkpoints, logs, and more)
# trainer = pl.Trainer(gpus=8) (if you have GPUs)
trainer = pl.Trainer()
trainer.fit(autoencoder, train_loader)

```Trainer```는 다음 사항을 자동화합니다.

- Epoch and batch iteration
- Calling of optimizer.step(), backward, zero_grad()
- Calling of .eval(), enabling/disabling grads
- weights loading
- Tensorboard (see loggers options)
- Multi-GPU support
- TPU
- 16-bit precision AMP support


>Tip: 
여러분이 만약 수동적으로 Optimizer들을 관리하는것을 선호한다면, 여러분은 the Manual optimization mode를 사용할 수 있습니다 (ie: RL, GANs, etc…).


---

**이게 전부입니다!**

다음은 Lightning에서 알아야 할 주요 2가지 개념입니다. Lightning의 다른 모든 기능은 Trainer 또는 LightningModule의 기능입니다.

---


## 기본적인 특징들
### Manual vs Automaitic optimization

#### Automatic optimization

Lightning을 사용하면, Training_step에서 Attached graph로 손실을 반환하는 한 grad를 활성화/비활성화하거나, Backward pass를 수행하거나, Optimizer를 업데이트할 때 걱정할 필요가 없습니다. Lightning이 최적화를 자동화합니다.  

In [ ]:
def training_step(self, batch, batch_idx):
    loss = self.encoder(batch)
    return loss

#### Manual optimization

하지만 특히 GAN같은 연구나 강화학습, 또는 여러개의 Optimizer를 사용하는 등의 경우에서는 여러분들이 Automatic optimization을 끄고 Training loop를 완전히 통제할 수 있습니다.

Automatic optimization을 끄고, 여러분이 Train loop를 통제해보세요!

In [ ]:
def __init__(self):
    self.automatic_optimization = False

def training_step(self, batch, batch_idx):
    # access your optimizers with use_pl_optimizer=False. Default is True
    opt_a, opt_b = self.optimizers(use_pl_optimizer=True)

    loss_a = self.generator(batch)
    opt_a.zero_grad()
    # use `manual_backward()` instead of `loss.backward` to automate half precision, etc...
    self.manual_backward(loss_a)
    opt_a.step()

    loss_b = self.discriminator(batch)
    opt_b.zero_grad()
    self.manual_backward(loss_b)
    opt_b.step()

---


### Predict or Deploy

여러분이 훈련을 수행할 때, Prediction을 위한 LightningModule의 3가지 옵션이 있습니다.

#### Option 1: Sub-models

예측을 위해 시스템 내부의 모든 모델을 꺼냅니다.

In [ ]:
# ----------------------------------
# to use as embedding extractor
# ----------------------------------
autoencoder = LitAutoEncoder.load_from_checkpoint('path/to/checkpoint_file.ckpt')
encoder_model = autoencoder.encoder
encoder_model.eval()

# ----------------------------------
# to use as image generator
# ----------------------------------
decoder_model = autoencoder.decoder
decoder_model.eval()

#### Option 2: Forward

원하는 대로 예측을 수행하기 위해 forward 메서드를 추가할 수도 있습니다.

In [ ]:
# ----------------------------------
# using the AE to extract embeddings
# ----------------------------------
class LitAutoEncoder(LightningModule):
    def __init__(self):
        super().__init__()
        self.encoder = nn.Sequential()

    def forward(self, x):
        embedding = self.encoder(x)
        return embedding

autoencoder = LitAutoEncoder()
autoencoder = autoencoder(torch.rand(1, 28 * 28))

#### Option 3: Production

Production system의 경우 onnx 또는 torchscript가 훨씬 빠릅니다. forward method를 추가했는지 확인하거나 필요한 sub-model만 추적하십시오.

In [ ]:
# ----------------------------------
# torchscript
# ----------------------------------
autoencoder = LitAutoEncoder()
torch.jit.save(autoencoder.to_torchscript(), "model.pt")
os.path.isfile("model.pt")

# ----------------------------------
# onnx
# ----------------------------------
with tempfile.NamedTemporaryFile(suffix='.onnx', delete=False) as tmpfile:
     autoencoder = LitAutoEncoder()
     input_sample = torch.randn((1, 28 * 28))
     autoencoder.to_onnx(tmpfile.name, input_sample, export_params=True)
     os.path.isfile(tmpfile.name)

---

### Using CPUs/GPUs/TPUs

Lightning에서 CPU, GPU 또는 TPU를 사용하는 것은 간단합니다. 코드를 변경할 필요가 없습니다. 트레이너 옵션만 변경하면 됩니다.

In [ ]:
# train on CPU
trainer = Trainer()

# train on 8 CPUs
trainer = Trainer(num_processes=8)

# train on 1024 CPUs across 128 machines
trainer = pl.Trainer(
    num_processes=8,
    num_nodes=128
)

# train on 1 GPU
trainer = pl.Trainer(gpus=1)

# train on multiple GPUs across nodes (32 gpus here)
trainer = pl.Trainer(
    gpus=4,
    num_nodes=8
)

코드의 한 줄을 변경하지 않고 이제 아래 코드로 구현할 수 있습니다.

In [ ]:
# train on TPUs using 16 bit precision
# using only half the training data and checking validation every quarter of a training epoch
trainer = pl.Trainer(
    tpu_cores=8,
    precision=16,
    limit_train_batches=0.5,
    val_check_interval=0.25
)

---

### Checkpoints

Lightning은 모델을 자동으로 저장합니다. 훈련을 마치면 다음과 같이 체크포인트를 로드할 수 있습니다.

In [ ]:
model = LitModel.load_from_checkpoint(path)

위의 체크포인트에는 모델을 초기화하고 상태 dict를 설정하는 데 필요한 모든 인수가 포함되어 있습니다. 수동으로 수행하려는 경우 아래 방법이 있습니다.

In [ ]:
# load the ckpt
ckpt = torch.load('path/to/checkpoint.ckpt')

# equivalent to the above
model = LitModel()
model.load_state_dict(ckpt['state_dict'])

---

### Data flow

각 loop (training, validation, test) 여러분이 구현할 수 있는 3개의 hook이 있습니다.
- x_step
- x_step_end
- x_epoch_end

다음의 예에서는 데이터 흐름을 설명하기 위해 훈련 루프(예: x=training)를 사용합니다.

In [ ]:
outs = []
for batch in data:
    out = training_step(batch)
    outs.append(out)
training_epoch_end(outs)

The equivalent in Lightning is:

In [ ]:
def training_step(self, batch, batch_idx):
    prediction = ...
    return prediction

def training_epoch_end(self, training_step_outputs):
    for prediction in predictions:
        # do something with these

DP 또는 DDP2 분산 모드를 사용하는 경우(예: GPU 간에 배치 분할) x_step_end를 사용하여 수동으로 집계합니다(또는 Lightning이 자동 집계하도록 구현하지 마십시오).

In [ ]:
for batch in data:
    model_copies = copy_model_per_gpu(model, num_gpus)
    batch_split = split_batch_per_gpu(batch, num_gpus)

    gpu_outs = []
    for model, batch_part in zip(model_copies, batch_split):
        # LightningModule hook
        gpu_out = model.training_step(batch_part)
        gpu_outs.append(gpu_out)

    # LightningModule hook
    out = training_step_end(gpu_outs)

Lightning에서는 다음과 같습니다.

In [ ]:
def training_step(self, batch, batch_idx):
    loss = ...
    return loss

def training_step_end(self, losses):
    gpu_0_loss = losses[0]
    gpu_1_loss = losses[1]
    return (gpu_0_loss + gpu_1_loss) * 1/2

> TIP: Valid loop와 Test loop는 동일한 구조입니다.

---

### Logging

Tensorboard나 여러분이 즐겨 사용하는 Logger 와/또는 Progress bar에 로깅하려면 LightningModule의 모든 메서드에서 호출할 수 있는 log() 메서드를 사용합니다.

In [ ]:
def training_step(self, batch, batch_idx):
    self.log('my_metric', x)

`log()` 메서드는 다음의 몇가지 옵션이 있습니다.

- on_step (logs the metric at that step in training)
- on_epoch (automatically accumulates and logs at the end of the epoch)
- prog_bar (logs to the progress bar)
- logger (logs to the logger like Tensorboard)

로그가 호출된 위치에 따라 Lightning은 올바른 모드를 자동으로 결정합니다. 그러나 물론 플래그를 수동으로 설정하여 기본 동작을 무시할 수 있습니다.

> NOTE: on_epoch=True로 설정하면 전체 훈련 에포크 동안 기록된 값이 누적됩니다.

In [ ]:
def training_step(self, batch, batch_idx):
    self.log('my_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)

> Note: Porgress bar에 표시된 손실 값은 마지막 값에 대해 평균되므로 학습/검증 단계에서 반환된 실제 손실과 다릅니다.

로거의 모든 방법을 직접 사용할 수도 있습니다.

In [ ]:
def training_step(self, batch, batch_idx):
    tensorboard = self.logger.experiment
    tensorboard.any_summary_writer_method_you_want())

훈련이 시작되면 즐겨 사용하는 로거를 사용하거나 Tensorboard 로그를 부팅하여 로그를 볼 수 있습니다.

```bash
tensorboard --logdir ./lightning_logs
```

> NOTE: Lightning은 Progress bar에 training_step에서 반환된 손실 값을 자동으로 표시합니다. 따라서 self.log('loss', loss, prog_bar=True)와 같이 명시적으로 기록할 필요가 없습니다.

자세한 사항은 [loggers](https://pytorch-lightning.readthedocs.io/en/latest/common/loggers.html) docs를 참고하세요.

---

### Optional extensions

#### Callbacks

콜백은 훈련 루프의 임의의 부분에서 실행할 수 있는 임의의 Self-contained program입니다.

다음은 그다지 화려하지 않은 Learning rate decay rule을 추가하는 예입니다.

In [ ]:
from pytorch_lightning.callbacks import Callback

class DecayLearningRate(Callback):

    def __init__(self):
        self.old_lrs = []

    def on_train_start(self, trainer, pl_module):
        # track the initial learning rates
        for opt_idx, optimizer in enumerate(trainer.optimizers):
            group = [param_group['lr'] for param_group in optimizer.param_groups]
            self.old_lrs.append(group)

    def on_train_epoch_end(self, trainer, pl_module, outputs):
        for opt_idx, optimizer in enumerate(trainer.optimizers):
            old_lr_group = self.old_lrs[opt_idx]
            new_lr_group = []
            for p_idx, param_group in enumerate(optimizer.param_groups):
                old_lr = old_lr_group[p_idx]
                new_lr = old_lr * 0.98
                new_lr_group.append(new_lr)
                param_group['lr'] = new_lr
            self.old_lrs[opt_idx] = new_lr_group

# And pass the callback to the Trainer
decay_callback = DecayLearningRate()
trainer = Trainer(callbacks=[decay_callback])

Callback으로 할 수 있는 것들:

- 훈련의 특정 지점에서 이메일 발송
- Grow the model
- Update learning rates
- Visualize gradients
- …
- 상상할 수 있는 모든것들

자세한 것은 [Callback](https://pytorch-lightning.readthedocs.io/en/latest/extensions/callbacks.html) docs를 참고하세요.

---


### LightningDataModules

DataLoader와 데이터 처리 코드는 결국 흩어지는 경향이 있습니다. LightningDataModule로 구성하여 데이터 코드를 재사용 가능하게 구현하세요.

In [ ]:
class MNISTDataModule(LightningDataModule):

      def __init__(self, batch_size=32):
          super().__init__()
          self.batch_size = batch_size

      # When doing distributed training, Datamodules have two optional arguments for
      # granular control over download/prepare/splitting data:

      # OPTIONAL, called only on 1 GPU/machine
      def prepare_data(self):
          MNIST(os.getcwd(), train=True, download=True)
          MNIST(os.getcwd(), train=False, download=True)

      # OPTIONAL, called for every GPU/machine (assigning state is OK)
      def setup(self, stage: Optional[str] = None):
          # transforms
          transform=transforms.Compose([
              transforms.ToTensor(),
              transforms.Normalize((0.1307,), (0.3081,))
          ])
          # split dataset
          if stage in (None, 'fit'):
              mnist_train = MNIST(os.getcwd(), train=True, transform=transform)
              self.mnist_train, self.mnist_val = random_split(mnist_train, [55000, 5000])
          if stage == (None, 'test'):
              self.mnist_test = MNIST(os.getcwd(), train=False, transform=transform)

      # return the dataloader for each split
      def train_dataloader(self):
          mnist_train = DataLoader(self.mnist_train, batch_size=self.batch_size)
          return mnist_train

      def val_dataloader(self):
          mnist_val = DataLoader(self.mnist_val, batch_size=self.batch_size)
          return mnist_val

      def test_dataloader(self):
          mnist_test = DataLoader(self.mnist_test, batch_size=self.batch_size)
          return mnist_test

LightningDataModule은 다양한 프로젝트에서 Data split 및 Transforms를 공유하고 재사용할 수 있도록 설계되었습니다. 다운로드, Tokenizing, Processing 등 데이터를 처리하는 데 필요한 모든 단계를 캡슐화합니다.

이제 LightningDataModule을 `Trainer`에 간단히 전달할 수 있습니다.

In [ ]:
# init model
model = LitModel()

# init data
dm = MNISTDataModule()

# train
trainer = pl.Trainer()
trainer.fit(model, dm)

# test
trainer.test(datamodule=dm)

DataModules는 특히 데이터를 기반으로 모델을 구축하는 데 유용합니다. 자세한 것은 [datamodules](https://pytorch-lightning.readthedocs.io/en/latest/extensions/datamodules.html) docs를 읽어보세요.

---

### Debugging

Lightning에는 디버깅을 위한 많은 도구가 있습니다. 다음은 그 중 일부의 예입니다.

In [ ]:
# use only 10 train batches and 3 val batches
trainer = Trainer(limit_train_batches=10, limit_val_batches=3)

In [ ]:
# Automatically overfit the sane batch of your model for a sanity test
trainer = Trainer(overfit_batches=1)

In [ ]:
# unit test all the code- hits every line of your code once to see if you have bugs,
# instead of waiting hours to crash on validation
trainer = Trainer(fast_dev_run=True)

In [ ]:
# train only 20% of an epoch
trainer = Trainer(limit_train_batches=0.2)

In [ ]:
# run validation every 25% of a training epoch
trainer = Trainer(val_check_interval=0.25)

In [ ]:
# Profile your code to find speed/memory bottlenecks
Trainer(profiler="simple")

---

## 나머지 멋진 특징들

첫 번째 Lightning 모델을 정의하고 교육한 후에는 다음과 같은 다른 멋진 기능을 사용해 볼 수 있습니다.

- [Automatic early stopping](https://pytorch-lightning.readthedocs.io/en/latest/common/early_stopping.html)
- [Automatic truncated-back-propagation-through-time](https://pytorch-lightning.readthedocs.io/en/latest/common/trainer.html#truncated-bptt-steps)
- [Automatically scale your batch size](https://pytorch-lightning.readthedocs.io/en/latest/advanced/training_tricks.html#auto-scaling-of-batch-size)
- [Automatically find a good learning rate](https://pytorch-lightning.readthedocs.io/en/latest/advanced/lr_finder.html)
- [Load checkpoints directly from S3](https://pytorch-lightning.readthedocs.io/en/latest/common/weights_loading.html#checkpoint-loading)
- [Scale to massive compute clusters](https://pytorch-lightning.readthedocs.io/en/latest/clouds/cluster.html)
- [Use multiple dataloaders per train/val/test loop](https://pytorch-lightning.readthedocs.io/en/latest/advanced/multiple_loaders.html)
- [Use multiple optimizers to do reinforcement learning or even GANs](https://pytorch-lightning.readthedocs.io/en/latest/common/optimizers.html#use-multiple-optimizers-like-gans)

혹은 우리의 [가이드](https://pytorch-lightning.readthedocs.io/en/latest/starter/introduction_guide.html)를 읽고 더 배워보세요!

----


### Grid AI

Grid AI는 클라우드에서 Large scale training 및 Tunning을 위한 기본 솔루션입니다.

[여기에서 GitHub 또는 Google 계정으로 무료로 시작해보세요!](https://www.grid.ai/)

---

### Community

우리 커뮤니티는 핵심 유지 관리자와 수천 명의 전문 연구원으로 구성되어 [Slack](https://pytorch-lightning.slack.com/join/shared_invite/zt-pw5v393p-qRaDgEk24~EjiZNBpSQFgQ#/shared-invite/email) 및 [GitHub Discussions](https://github.com/PyTorchLightning/pytorch-lightning/discussions)에서 활발히 활동하고 있습니다. 놀러 가거나 Lightning 질문을 하거나 연구에 대해 토론할 수도 있습니다!

---


### Masterclass

또한 Lightning의 고급 사용법을 가르치는 마스터 클래스도 제공합니다.
[![pl](https://pytorch-lightning.readthedocs.io/en/latest/_images/PTL101_youtube_thumbnail.jpg)](https://www.youtube.com/playlist?list=PLaMu-SDt_RB5NUm67hU2pdE75j6KaIOv2)